In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import torch
from torch import nn
from torchvision import transforms
from collections import Counter



In [2]:
transform  =transforms.Compose([
    transforms.Resize((500,500)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [3]:
device =  "cuda" if torch.cuda.is_available() else 'cpu'

In [4]:
moves = {0: 'None', 1: 'jump', 2: 'left', 3: 'right', 4: 'roll'}

In [5]:
class SubWayNet(nn.Module):

    def __init__(self):

        # (472, 988)
        super(SubWayNet,self).__init__()
        # self.conv1 =  
        # self.conv2 =  nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        # self.conv3 =  nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        # self.conv4 =  nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)

        self.sequential1 =  nn.Sequential(
            nn.Conv2d(in_channels=3,stride=3, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.sequential2 =  nn.Sequential(
            nn.Conv2d(in_channels=64, stride=3, out_channels=128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            # nn.Dropout(p=0.5)
        )

        # self.sequential3 =  nn.Sequential(
        #             nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
        #             nn.BatchNorm2d(128),
        #             nn.ReLU(),
        #             nn.MaxPool2d(kernel_size=2),
        #             nn.Dropout(p=0.5)
        # )

    
        # THIS ASSUMES THAT THE SEQUENTIAL IS 500
        self.fc_layers =  nn.Sequential(
                nn.Linear(128*13*13,64),
                nn.ReLU(),
                nn.Linear(64,5)
        )

    def forward(self,x):

        x =  self.sequential1(x)
        x =  self.sequential2(x)
        # x =  self.sequential3(x)
        # x = x.view(x.size(0),-1)
        x= x.view(-1,128*13*13)
     

        x =  self.fc_layers(x)
        return x

In [12]:
# class SubWayNet(nn.Module):

#     def __init__(self):

#         # (472, 988)
#         super(SubWayNet,self).__init__()
#         # self.conv1 =  
#         # self.conv2 =  nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
#         # self.conv3 =  nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
#         # self.conv4 =  nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)

#         self.sequential1 =  nn.Sequential(
#             nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2)
#         )

#         self.sequential2 =  nn.Sequential(
#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2)
#         )

#         self.sequential3 =  nn.Sequential(
#                     nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
#                     nn.BatchNorm2d(128),
#                     nn.ReLU(),
#                     nn.MaxPool2d(kernel_size=2)
#         )

    
#         # THIS ASSUMES THAT THE SEQUENTIAL IS 500
#         self.fc_layers =  nn.Sequential(
#                 nn.Linear(128*60*60,64),
#                 nn.ReLU(),
#                 nn.Linear(64,5)
#         )

#     def forward(self,x):

#         x =  self.sequential1(x)
#         x =  self.sequential2(x)
#         x =  self.sequential3(x)
#         x = x.view(-1, 128*60*60)

#         x =  self.fc_layers(x)
#         return x

In [6]:
model=SubWayNet()

In [7]:
model =  torch.load("finalTrial_complete_data.pt")

In [8]:
model.to(device)

SubWayNet(
  (sequential1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (sequential2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(3, 3))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=21632, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=5, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)

In [9]:
def prepare(filename:str):
    
    pt =  f"TEST_S/{filename}.jpg"
    img = Image.open(pt)
    
    return  transform(img).unsqueeze(0).to(device)

In [12]:
a = prepare('1')

In [10]:
a = prepare(str(1))
model.eval()
with torch.no_grad():
    logits = model(a)
_,pred = torch.max(logits, 1)
print(moves[int(pred)])

right


In [20]:
aa= []
for i in range(1,52):
        
    a = prepare(str(i))
    model.eval()
    with torch.no_grad():
        logits = model(a)
    _,pred = torch.max(logits, 1)
    # print(moves[int(pred)])
    aa.append(moves[int(pred)])

In [18]:
set(aa)
    

{(1, 'right'),
 (2, 'right'),
 (3, 'right'),
 (4, 'right'),
 (5, 'right'),
 (6, 'right'),
 (7, 'left'),
 (8, 'right'),
 (9, 'right'),
 (10, 'left'),
 (11, 'right'),
 (12, 'None'),
 (13, 'None'),
 (14, 'left'),
 (15, 'None'),
 (16, 'None'),
 (17, 'left'),
 (18, 'None'),
 (19, 'right'),
 (20, 'right'),
 (21, 'right'),
 (22, 'right'),
 (23, 'right'),
 (24, 'right'),
 (25, 'right'),
 (26, 'right'),
 (27, 'right'),
 (28, 'right'),
 (29, 'right'),
 (30, 'right'),
 (31, 'right'),
 (32, 'None'),
 (33, 'None'),
 (34, 'None'),
 (35, 'left'),
 (36, 'left'),
 (37, 'jump'),
 (38, 'roll'),
 (39, 'roll'),
 (40, 'left'),
 (41, 'jump'),
 (42, 'jump'),
 (43, 'jump'),
 (44, 'jump'),
 (45, 'jump'),
 (46, 'None'),
 (47, 'None'),
 (48, 'left'),
 (49, 'right'),
 (50, 'right'),
 (51, 'right')}

In [50]:
moves

{0: 'None', 1: 'jump', 2: 'left', 3: 'right', 4: 'roll'}

In [87]:
set(aa)

{'None', 'left', 'right', 'roll'}

In [21]:
Counter(aa)

Counter({'right': 25, 'None': 10, 'left': 8, 'jump': 6, 'roll': 2})